In [1]:
from gurobipy import *
import pandas as pd
import numpy as np

# Constants
T = int(1440 / 5)
K = 10
V = [0, 1]

# Create a new model
m = Model("Vertiport_Aircraft_Routing")

# Create variables
ni = m.addVars(((t, i, k) for t in range(T+1) for i in V for k in range(K+1)), vtype=GRB.INTEGER, name="n")
uij = m.addVars(((t, i, j, k) for t in range(T+1) for i in V for j in V for k in range(K+1) if i != j), vtype=GRB.INTEGER, name="u")
cijk = m.addVars(((t, i, x, y) for t in range(T+1) for i in V for x in range(K+1) for y in range(K+1) if x < y), vtype=GRB.INTEGER, name="c")

# rest of your initializations...


# Flight time matrix
tau = [[0, 6.06667 / 10, 0], [6.06667 / 10, 0, 6.06667 / 10], [0, 6.06667 / 10, 0]]

# SOC levels drop matrix
kappa = [[0, -0.8 / 10, 0], [-0.8 / 10, 0, -0.8 / 10], [0, -0.8 / 10, 0]]

gamma = [3.047619048 / 10, 3.047619048 / 10, 3.237872699 / 10, 3.729470167 / 10, 4.404786588 / 10, 5.379957014 / 10, 6.913363091 / 10,
        9.685271742 / 10, 16.30528373 / 10, 71.41103553 / 10]

# f

f_values = np.zeros((T, 2, 2))
data = pd.read_csv('input/schedule.csv')
LAX_DTLA = data[data['od'] == 'LAX_DTLA']
DTLA_LAX = data[data['od'] == 'DTLA_LAX']
# Create the list of lists
LAX_DTLA = [[1 if i in LAX_DTLA['schedule'].tolist() else 0] for i in range(1440)]
DTLA_LAX = [[1 if i in DTLA_LAX['schedule'].tolist() else 0] for i in range(1440)]

# Reshape the array
new_array_LAX_DTLA = np.array(LAX_DTLA).reshape((144, 10))
new_array_DTLA_LAX = np.array(DTLA_LAX).reshape((144, 10))
# Convert back to list
new_array_LAX_DTLA = new_array_LAX_DTLA.tolist()
new_array_DTLA_LAX = new_array_DTLA_LAX.tolist()
# Add elements within each cell
new_array_LAX_DTLA_sum = np.sum(new_array_LAX_DTLA, axis=1)
new_array_DTLA_LAX_sum = np.sum(new_array_DTLA_LAX, axis=1)

LAX_DTLA = new_array_LAX_DTLA_sum
DTLA_LAX = new_array_DTLA_LAX_sum

for t in range(T):
    f_values[t][0][1] = LAX_DTLA[t] # get the first (and only) item of the inner list
    f_values[t][1][0] = DTLA_LAX[t] # get the first (and only) item of the inner list
    
# Define the objective
m.setObjective(ni.sum(0, '*', '*') + uij.sum(0, '*', '*', '*') + cijk.sum(0, '*', '*', '*'), GRB.MINIMIZE)


# Constraints
# Stationarity
for i in V:
    for k in range(K+1):
        m.addConstr(ni[0, i, k] + uij.sum(0, i, '*', k) + cijk.sum(0, i, k, '*') == ni[T, i, k] + uij.sum(T, i, '*', k) + cijk.sum(T, i, k, '*'))

# The flight schedule must be satisfied.
for j in [0, 1]:
    for i in [0, 1]:
        if i != j:
            for t in range(T):
                m.addConstr(uij.sum(t, i, j, '*') >= f_values[t][i][j])

# Can't fly when SOC = 0
for i in V:
    for j in V:
        if i != j:
            m.addConstr(uij[0, i, j, 0] == 0)

# rest of your constraints...

# Dynamic equation
for i in V:
    for k in range(K+1):
        for t in range(T+1):
            if t > 0:
                m.addConstr(
                    ni[t, i, k] == ni[t-1, i, k] + 
                    quicksum(uij[t-1-tau[j][i], j, i, k+kappa[j][i]] for j in V if j != i and t-1-tau[j][i] >= 0 and (t-1-tau[j][i], j, i, k+kappa[j][i]) in uij) -
                    quicksum(uij[t-1, i, j, k] for j in V if j != i and (t-1, i, j, k) in uij) +
                    quicksum(cijk[t-int(sum(gamma[x:k])), i, x, k] for x in range(k)) -
                    quicksum(cijk[t, i, k, y] for y in range(K+1) if y > k)
                )

# Optimize model
m.optimize()

# Integrate new variables
m.update()

# Solve model
m.optimize()
# Redirect stdout to a file
sys.stdout = open('output.txt', 'w')

if m.status == GRB.Status.INFEASIBLE:
    print('The model is infeasible; computing IIS')
    m.computeIIS()
    m.write("model.ilp")
    m.feasRelaxS(0, False, False, True) # calculate relaxed solution
    m.optimize()





# Print results
for v in m.getVars():
    if v.x > 0:  # Print only non-zero variables for clarity
        print('{} = {}'.format(v.varName, v.x))

print('Total Fleet Size:', m.objVal)

# Close the output file
sys.stdout.close()



Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-21
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-10910 CPU @ 3.60GHz
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 14184 rows, 14355 columns and 61457 nonzeros
Model fingerprint: 0x3d4d6af9
Variable types: 0 continuous, 14355 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 80 rows and 4917 columns
Presolve time: 0.01s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.01 work units)
Thread count was 1 (of 20 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
